### 加载环境和基础设置

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import time
import logging
import glob
import subprocess
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from datetime import datetime
import shutil

# Import necessary classes from separate files
from MacGenerator import EnergySpectrum, MacFileGenerator, set_energy_decimal_places
from RootReader import RootData
from MySim import MySim # Import the new MySim class

# 切换到当前目录，并获取工作目录
try:
    os.chdir(sys.path[0])
except FileNotFoundError:
    # Handle cases where sys.path[0] might not be a valid directory (e.g., interactive environments)
    print(f"警告: 无法切换到目录 {sys.path[0]}, 使用当前工作目录。")
    pass # Continue with the current working directory

current_path = os.path.abspath(os.curdir)
# 把r'../../build' 加入到sys.path中 (确保路径正确)
build_path = os.path.abspath(os.path.join(current_path, r'../build'))
if build_path not in sys.path:
    sys.path.append(build_path)

# 定义相对路径
rel_dir_root = r'./Data/RawData/'  # ROOT files directory
rel_dir_csv  = r'./Data/CSVData/'  # CSV output directory
rel_dir_mac  = r'./Data/MacLog/'   # MAC files directory
rel_dir_log  = r'./Data/RunLog/'   # Simulation log directory
rel_dir_geant4_executable = r'../build/CompScintSim' # Geant4 executable relative path

# 获取绝对路径，末尾加上 '/'. 使用 os.path.normpath 确保路径格式一致
abs_dir_root   = os.path.normpath(os.path.join(current_path, rel_dir_root)) + os.sep
abs_dir_csv    = os.path.normpath(os.path.join(current_path, rel_dir_csv)) + os.sep
abs_dir_mac    = os.path.normpath(os.path.join(current_path, rel_dir_mac)) + os.sep
abs_dir_log    = os.path.normpath(os.path.join(current_path, rel_dir_log)) + os.sep
abs_dir_geant4_executable = os.path.normpath(os.path.join(current_path, rel_dir_geant4_executable))

print("RootDir:  ", abs_dir_root)
print("CSVDir:   ", abs_dir_csv)
print("MacDir:   ", abs_dir_mac)
print("LogDir:   ", abs_dir_log)
print("Geant4 Exe:", abs_dir_geant4_executable)

# 确保各个目录存在
for directory in [abs_dir_root, abs_dir_csv, abs_dir_mac, abs_dir_log]:
    os.makedirs(directory, exist_ok=True)

# ---- 日志配置 -----
log_formatter = logging.Formatter('%(asctime)s - %(levelname)s [%(filename)s:%(lineno)d] %(message)s')
log_file_handler = None
def setup_logging():
    """配置日志记录，确保只输出到文件，并阻止控制台输出。"""
    global log_file_handler
    
    # 获取根记录器
    root_logger = logging.getLogger()
    
    # --- 1. 严格清理现有处理器 ---
    # 移除所有当前附加到根记录器的处理器
    existing_handlers = root_logger.handlers[:]
    for handler in existing_handlers:
        try:
            if hasattr(handler, 'close'):
                handler.close()
        except Exception:
            pass
        root_logger.removeHandler(handler)
    log_file_handler = None # 重置跟踪器
    
    # --- 2. 添加唯一的文件处理器 ---
    log_filename = os.path.join(abs_dir_log, "0_batch_process.log")
    try:
        # 创建文件处理器
        log_file_handler = logging.FileHandler(log_filename, mode='a', encoding='utf-8') # 追加模式
        log_file_handler.setFormatter(log_formatter)
        root_logger.addHandler(log_file_handler)
        
        # 添加空处理器以防止信息传播到控制台
        null_handler = logging.NullHandler()
        root_logger.addHandler(null_handler)
        
        # 重要：设置级别和禁止传播
        root_logger.setLevel(logging.INFO)
        root_logger.propagate = False
        
        # 禁用常见第三方库的日志输出
        for logger_name in ['matplotlib', 'IPython', 'jupyter', 'tornado', 'traitlets', 'parso']:
            third_party_logger = logging.getLogger(logger_name)
            third_party_logger.setLevel(logging.WARNING)
            third_party_logger.propagate = False
            # 清理该记录器上的所有处理器
            for h in third_party_logger.handlers[:]:
                third_party_logger.removeHandler(h)
            third_party_logger.addHandler(logging.NullHandler())
        
        # 写入初始日志记录
        logging.info("=" * 50)
        logging.info("初始化批处理日志 (仅文件) - %s", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        logging.info("Log file: %s", log_filename)
        
    except Exception as e:
        print(f"!!!!!! 错误：无法设置日志文件 {log_filename}: {e} !!!!!!")
        print("!!!!!! 日志记录将不可用。 !!!!!!")


# 确保在脚本/单元格开始时调用
setup_logging() # Initial setup 

# 全局变量，记录批处理开始前的数据目录文件列表 
global_before_files = set(os.listdir(abs_dir_root)) 

# 设置能量显示小数位数 (来自 MacGenerator) 
set_energy_decimal_places(2) 

# ---- Geant4 运行函数 ---- 

def run_geant4(dir_geant4_exe, dir_mac_file, log_file_base, retries=3): 
    """ 
    运行 Geant4 模拟，并记录详细日志。 
    - Geant4 标准输出/错误输出写入单独的模拟日志文件。 
    - 在主批处理日志中记录模拟状态、耗时、新文件等。 
    - 处理失败重试逻辑。 

    参数: 
        dir_geant4_exe (str): Geant4 可执行文件完整路径。 
        dir_mac_file (str): 要执行的 MAC 文件完整路径。 
        log_file_base (str): 模拟日志文件的基础名 (不含路径和扩展名)。 
        retries (int): 失败后的重试次数。 

    返回: 
        list: 模拟成功时生成的新 ROOT 文件列表 (相对于 abs_dir_root)。失败时返回空列表。 
    """ 
    global global_before_files 

    mac_filename = os.path.basename(dir_mac_file) 
    logging.info(f"--- 开始运行 Geant4 [MAC: {mac_filename}] ---") 
    start_time = time.time() 

    # 检查 Geant4 可执行文件是否存在 
    if not os.path.isfile(dir_geant4_exe): 
        logging.error(f"Geant4 可执行文件未找到: {dir_geant4_exe}") 
        raise FileNotFoundError(f"Geant4 可执行文件未找到: {dir_geant4_exe}") 

    # 设置模拟特定日志文件路径 
    simulation_log_file = os.path.join(abs_dir_log, f"{log_file_base}.log") 

    # 运行 Geant4 并将 stdout 和 stderr 重定向到模拟日志文件 
    process = None 
    try: 
        with open(simulation_log_file, "w") as log_f: 
            # Use Popen for better control, especially if needing timeouts later 
            cmd = [dir_geant4_exe, "-m", dir_mac_file] 
            logging.info(f"执行命令: {' '.join(cmd)}") 
            process = subprocess.Popen(cmd, stdout=log_f, stderr=subprocess.STDOUT, cwd=os.path.dirname(dir_geant4_exe)) 
            process.wait() # Wait for the process to complete 

            if process.returncode != 0: 
                logging.warning(f"Geant4 进程返回非零代码: {process.returncode}. 查看日志: {simulation_log_file}") 
            else: 
                logging.debug(f"Geant4 进程成功完成 (返回码 0)。") 

    except FileNotFoundError: 
        logging.error(f"命令执行失败: Geant4 可执行文件 '{dir_geant4_exe}' 或 MAC 文件 '{dir_mac_file}' 不存在或无权限。") 
        return [] # Indicate failure 
    except Exception as e: 
        logging.error(f"运行 Geant4 时发生未知错误: {e}", exc_info=True) 
        if process and process.poll() is None: # Check if process is still running 
            process.terminate() # Try to terminate if hung 
        return [] # Indicate failure 

    elapsed_time = time.time() - start_time 
    logging.info(f"Geant4 进程执行完毕，耗时: {elapsed_time:.2f} 秒。") 

    # 检查新生成的文件 
    after_files = set(os.listdir(abs_dir_root)) 
    new_files = list(after_files - global_before_files) 
    new_root_files = [f for f in new_files if f.endswith('.root')] 

    logging.info(f"在 {abs_dir_root} 中检测到新文件: {new_files or '无'}") 

    # 检查是否有指示失败的 '_t0' 文件 
    failed_t0_files = [f for f in new_root_files if "_t0" in f] 
    failed = bool(failed_t0_files) 

    if failed: 
        logging.error(f"模拟失败: 检测到失败标记文件 {failed_t0_files}。") 
        # 删除所有可能相关的 *_t*.root 文件 
        pattern = os.path.join(abs_dir_root, "*_t*.root") 
        deleted_count = 0 
        for file_to_delete in glob.glob(pattern): 
            if file_to_delete in after_files: # Only delete if it's potentially from this run 
                try: 
                    os.remove(file_to_delete) 
                    logging.info(f"删除失败的输出文件: {file_to_delete}") 
                    deleted_count += 1 
                except OSError as e: 
                    logging.warning(f"无法删除文件 {file_to_delete}: {e}") 

        # 更新文件列表，排除已删除的文件 
        current_files = set(os.listdir(abs_dir_root)) 
        global_before_files = current_files # Reset before retry 

        if retries > 0: 
            logging.warning(f"尝试重新运行模拟... 剩余重试次数: {retries-1}") 
            return run_geant4(dir_geant4_exe, dir_mac_file, log_file_base, retries=retries-1) 
        else: 
            logging.error("模拟失败，已达到最大重试次数。") 
            return [] # Failed after retries 
    else: 
        # 检查是否生成了预期的 ROOT 文件 (基于 MAC 文件名) 
        expected_root_basename = mac_filename.replace('.mac', '.root') 
        # Allow for potential timestamp additions if the original file existed 
        expected_root_found = any(f.startswith(mac_filename.replace('.mac', '')) and f.endswith('.root') for f in new_root_files) 

        if not new_root_files: 
            logging.warning("模拟似乎成功 (无 _t0 文件)，但未在 {abs_dir_root} 中找到新的 .root 文件。") 
        elif not expected_root_found: 
             logging.warning(f"模拟似乎成功，但找到的 ROOT 文件 {new_root_files} 与预期 ({expected_root_basename}) 不完全匹配。") 
        else: 
            logging.info(f"模拟成功！[MAC: {mac_filename}] 耗时: {elapsed_time:.2f} 秒. 新 ROOT 文件: {new_root_files}") 

        # 更新全局文件列表以便下次对比 
        global_before_files = after_files 
        return new_root_files # Return list of new root files relative to abs_dir_root 

# ---- 辅助函数：清理目录 ---- 

def clear_data_directories(): 
    """清理并重建Data目录""" 
    directories_to_clear = [abs_dir_root, abs_dir_csv, abs_dir_mac, abs_dir_log] 
    for directory in directories_to_clear: 
        try: 
            if os.path.exists(directory): 
                # Close log file handler before deleting log directory 
                if directory == abs_dir_log and log_file_handler: 
                    log_file_handler.close() 
                    logging.root.removeHandler(log_file_handler) 

                shutil.rmtree(directory) 

            os.makedirs(directory, exist_ok=True) 

        except Exception as e: 
            logging.error(f"清理或创建目录 {directory} 时出错: {e}", exc_info=True) 


    # Reset global file list after clearing 
    global global_before_files 
    global_before_files = set(os.listdir(abs_dir_root)) # Update after clearing 

 # ---- 批量运行函数 ----
def run_batch_simulations(configs, batch_size):
    """
    根据提供的配置列表运行批量模拟。
    日志只输出到文件，控制台只保留 tqdm 进度条和最终总结。

    参数:
        configs (list): 包含多个模拟配置字典的列表。
        batch_size (int): 每个配置要运行的模拟次数。
    """
    if not isinstance(configs, list):
        configs = [configs]

    total_simulations = len(configs) * batch_size
    if total_simulations == 0:
        logging.warning("未提供配置或 batch_size 为 0，不执行模拟。")
        print("未执行模拟。")
        return

    # --- 日志记录只进入文件 ---
    logging.info(f"--- 开始批量模拟 ---")
    logging.info(f"配置数量: {len(configs)}")
    logging.info(f"每个配置的运行次数 (Batch): {batch_size}")
    logging.info(f"总模拟运行次数: {total_simulations}")
    # -------------------------

    global_sim_counter = 0
    successful_runs = 0
    failed_runs = 0

    # 检查可执行文件
    if not os.path.isfile(abs_dir_geant4_executable):
         logging.error(f"Geant4 可执行文件未找到: {abs_dir_geant4_executable}. 无法启动。")
         print(f"错误：Geant4 可执行文件未找到: {abs_dir_geant4_executable}") # 这个错误信息允许打印到控制台
         return

    # --- 直接使用 tqdm 循环 ---
    with tqdm(total=total_simulations, desc="运行 Geant4 模拟", unit="run", leave=True) as pbar: # leave=True 保留完成后的进度条
        for config_index, config in enumerate(configs):
            base_profile = config.get('profile', f'config_{config_index+1}')
            logging.info(f"==> 处理配置 {config_index + 1}/{len(configs)}: '{base_profile}' ===") # 只进文件

            for i in range(batch_size):
                global_sim_counter += 1
                run_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                run_profile_name = f"{base_profile}_b{i+1}_{run_timestamp}"
                run_log_base = run_profile_name

                pbar.set_description(f"运行: {run_profile_name}") # 更新进度条描述

                try:
                    # --- 准备配置并计算 beamOn ---
                    current_config = config.copy()
                    current_config['profile'] = run_profile_name

                    mode = current_config.get('mode')
                    gps_mode = current_config.get('gps_mode', 'native' if mode == 'weighted' else 'custom')

                    if mode == 'weighted' and gps_mode == 'native':
                        try:
                             calculated_num_events = EnergySpectrum.calculate_native_beam_on(current_config)
                             original_num_events = current_config.get('num_events')
                             if calculated_num_events != original_num_events:
                                 logging.info(f"原生加权模式: 自动计算 beamOn = {calculated_num_events} (覆盖配置值 {original_num_events})") # 只进文件
                             current_config['num_events'] = calculated_num_events
                        except Exception as calc_e:
                             logging.error(f"计算原生 beamOn 失败: {calc_e}. 使用原始或默认 num_events。", exc_info=True) # 只进文件
                             if 'num_events' not in current_config:
                                 logging.warning("配置无 num_events 且计算失败, 使用默认值 100。") # 只进文件
                                 current_config['num_events'] = 100

                    # --- 创建 MySim 实例 ---
                    sim = MySim.from_config(current_config)
                    logging.info(f"使用 MySim: {sim.get_profile()} (ROOT: {sim.root_file})") # 只进文件

                    # --- 写 MAC 文件 ---
                    mac_file_path = sim.write_mac_file(abs_dir_mac)
                    logging.info(f"生成 MAC: {os.path.basename(mac_file_path)}") # 只进文件

                    # --- 运行 Geant4 ---
                    new_root_files = run_geant4(abs_dir_geant4_executable, mac_file_path, run_log_base)

                    # --- 处理 ROOT 文件 ---
                    if new_root_files:
                        successful_runs += 1
                        logging.info(f"运行成功 ({run_profile_name}), 找到 {len(new_root_files)} 个 ROOT 文件。") # 只进文件
                        # ... (省略处理 ROOT 的详细循环，假设内部日志也只进文件) ...
                        for root_file_name in new_root_files:
                             if not root_file_name.startswith(run_profile_name): continue
                             # ... (实际处理逻辑) ...
                    else:
                        failed_runs += 1
                        sim_log_path = os.path.join(abs_dir_log, f"{run_log_base}.log")
                        logging.error(f"运行失败 ({run_profile_name})。查看: {sim_log_path}") # 只进文件

                except Exception as e:
                    # 捕捉循环内其他错误
                    failed_runs += 1
                    logging.error(f"运行 {run_profile_name} 时发生错误: {e}", exc_info=True) # 只进文件

                # 更新进度条
                pbar.update(1)
            # 结束单个配置批处理
        # 结束所有配置循环

    # --- 最终总结日志（只进入文件）---
    logging.info("--- 批量模拟完成 ---")
    logging.info(f"总运行次数: {global_sim_counter}")
    logging.info(f"成功次数: {successful_runs}")
    logging.info(f"失败次数: {failed_runs}")
    # --- 最终打印到控制台 ---
    print(f"\n批量模拟完成! 成功: {successful_runs}, 失败: {failed_runs}. 详情请查看日志文件。")


def process_all_root_files():
    """处理RawData目录下的所有.root文件并保存为CSV"""
    root_files = [f for f in os.listdir(abs_dir_root) if f.endswith('.root')]
    
    print(f"\n发现{len(root_files)}个root文件，开始处理...")
    
    with tqdm(total=len(root_files), desc="处理root文件", unit="file") as pbar:
        for root_file_name in root_files:
            try:
                root_file_path = os.path.join(abs_dir_root, root_file_name)
                root_data = RootData(root_file_path)
                
                # 生成CSV文件名（基于root文件名和时间戳）
                base_name = os.path.splitext(root_file_name)[0]
                csv_filename = f"{base_name}"
                
                # 保存CSV文件
                csv_path = root_data.save_simulation_data(abs_dir_csv, csv_filename)
                logging.info(f"处理完成: {root_file_path} -> {csv_path}")
            
            except Exception as e:
                logging.error(f"处理文件 {root_file_name} 时出错: {str(e)}")
            
            pbar.update(1)
    
    print(f"完成处理所有root文件！")

### 示例：运行批量模拟 (新配置模式)

In [ ]:
# 1. 清理旧数据 (可选) 
clear_data_directories() 

# 2. 定义模拟配置列表 

# 配置 1: Range Mode (自定义源) 
config_range = { 
    'mode': 'range',             # 模式: 能量范围 + 随机粒子数 
    'profile': 'electron_range_sim', # 文件名前缀 
    'gps_mode': 'custom',        # 使用自定义 /gps/my_source/add 
    'num_events': 20,           # Geant4 /run/beamOn 次数 

    # 电子配置 
    'E_e': [0.1, 2.0],           # 电子能量范围 [min, max] MeV 
    'delta_E_e': 0.2,            # 电子能量步长 MeV 
    'N_e_once_min': 5,           # 每个能量点单次事件最少电子数 
    'N_e_once_max': 15,          # 每个能量点单次事件最多电子数 

    # 质子配置 (可选) 
    'E_p': [5.0, 10.0], 
    'delta_E_p': 1.0, 
    'N_p_once_min': 0, 
    'N_p_once_max': 1, 
} 

# 配置 2: Weighted Mode (原生 GPS) 
config_weighted = { 
    'mode': 'weighted',          # 模式: 指定能量点 + 权重 
    'profile': 'proton_weighted_sim', # 文件名前缀 
    'gps_mode': 'native',        # 使用原生 /gps/source/add 

    # 电子配置
    'E_e': [0.5,1, 1.5,2], 
    'weights_e': [8,2.0, 1.0,3], 
    'N_e_once_min': 50,          # 权重最小 (1.0) 的能量点对应的基础粒子数基准 
    'scale_factor': 2,
} 

# 配置 3: Single Particle Mode (方便调试或特定测试) 
config_single = { 
    'mode': 'single', 
    'profile': 'single_gamma_1MeV', 
    'gps_mode': 'custom', # 或者 'native' 
    'num_events': 1000, 
    'particle': 'gamma', 
    'energy': 1.0,        # MeV 
    'count': 1            # 每个事件产生1个该粒子 
} 

# 配置 4: 旧版随机能谱模式 (通过嵌套配置) 
config_random_legacy = { 
    'mode': 'random', # 使用新的包装器模式 
    'profile': 'legacy_random_test', 
    'gps_mode': 'custom', # 或 'native' 
    'num_events': 50, 
    'random_config': { # 嵌套旧的随机生成器配置 
        'e_energy_min': 0.5, 
        'e_energy_max': 3.0, 
        'e_types_min': 2, 
        'e_types_max': 3, 
        'p_types_min': 1, 
        'p_types_max': 2, 
        'g_types_min': 0, # 不生成伽马 
        'g_types_max': 0, 
        'use_gamma': False, 
        'e_count_min': 10, 
        'e_count_max': 25, 
        'p_count_min': 5, 
        'p_count_max': 15, 
        # 'nums' is implicitly handled by top-level 'num_events' 
    } 
} 

# 3. 设置批量运行参数 
simulation_configs = [config_range, config_weighted, config_single] # 要运行的配置列表 
runs_per_config = 1  # 每个配置运行多少次 (Batch size) 

# 4. 运行批量模拟 
run_batch_simulations(simulation_configs, runs_per_config) 



In [7]:
clear_data_directories() 

# 配置 1: Range Mode (自定义源) 
test1_config = { 
    'mode': 'range',             # 模式: 能量范围 + 随机粒子数 
    'profile': 'electron_range_sim', # 文件名前缀 
    'gps_mode': 'custom',        # 使用自定义 /gps/my_source/add 
    'num_events': 20,           # Geant4 /run/beamOn 次数 

    # 电子配置 
    'E_e': [0.1, 2.0],           # 电子能量范围 [min, max] MeV 
    'delta_E_e': 0.2,            # 电子能量步长 MeV 
    'N_e_once_min': 5,           # 每个能量点单次事件最少电子数 
    'N_e_once_max': 15,          # 每个能量点单次事件最多电子数 
} 


test2_config = { 
    'mode': 'weighted',          # 模式: 指定能量点 + 权重 
    'profile': 'electron_weighted_sim', # 文件名前缀 
    'gps_mode': 'native',        # 使用原生 /gps/source/add 
 
    # 电子配置
    'E_e': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0], 
    'weights_e': [1,2,3,4,5,6,7,8,9,10,10,9,8,7,6,5,4,3,2,1], 
    'N_e_once_min': 10000,          # 权重最小 (1.0) 的能量点对应的基础粒子数基准 
    'scale_factor': 3, # 最终生成粒子数乘以的比例范围
} 
 
simulation_configs = [test2_config] # 要运行的配置列表 
runs_per_config = 3  # 每个配置运行多少次 (Batch size) 

# 运行批量模拟 
run_batch_simulations(simulation_configs, runs_per_config) 
process_all_root_files()

运行: electron_weighted_sim_b3_20250406_160939: 100%|██████████| 3/3 [00:51<00:00, 17.13s/run]



批量模拟完成! 成功: 3, 失败: 0. 详情请查看日志文件。

发现3个root文件，开始处理...


处理root文件: 100%|██████████| 3/3 [00:04<00:00,  1.52s/file]

完成处理所有root文件！
